In [1]:
#Arbol - Librerias
!conda install -c etetoolkit ete3

!export PATH=~/anaconda_ete/bin:$PATH

!conda install -c bioconda iqtree


Solving environment: - 
  - anaconda/linux-64::ca-certificates-2020.6.24-0, anaconda/linux-64::certifi-2020.6.20-py37_0, anaconda/linux-64::conda-4.8.3-py37_0, anaconda/linux-64::openssl-1.1.1g-h7b6447c_0
  - anaconda/linux-64::ca-certificates-2020.6.24-0, anaconda/linux-64::certifi-2020.6.20-py37_0, anaconda/linux-64::openssl-1.1.1g-h7b6447c_0, defaults/linux-64::conda-4.8.3-py37_0
  - anaconda/linux-64::certifi-2020.6.20-py37_0, anaconda/linux-64::conda-4.8.3-py37_0, anaconda/linux-64::openssl-1.1.1g-h7b6447c_0, defaults/linux-64::ca-certificates-2020.6.24-0
  - anaconda/linux-64::certifi-2020.6.20-py37_0, anaconda/linux-64::openssl-1.1.1g-h7b6447c_0, defaults/linux-64::ca-certificates-2020.6.24-0, defaults/linux-64::conda-4.8.3-py37_0
  - anaconda/linux-64::conda-4.8.3-py37_0, anaconda/linux-64::openssl-1.1.1g-h7b6447c_0, defaults/linux-64::ca-certificates-2020.6.24-0, defaults/linux-64::certifi-2020.6.20-py37_0
  - anaconda/linux-64::openssl-1.1.1g-h7b6447c_0, defaults/linux-64::ca

In [2]:
#Mapa - libreria
!pip install geopy
!pip install folium
!pip install selenium

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="TP FINAL Bioinformatica")

In [3]:
! conda install -c anaconda pillow
! apt-get install -y python-tk
! apt-get install -y clustalo


Solving environment: done

# All requested packages already installed.

E: No se pudo abrir el fichero de bloqueo «/var/lib/dpkg/lock-frontend» - open (13: Permiso denegado)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
E: No se pudo abrir el fichero de bloqueo «/var/lib/dpkg/lock-frontend» - open (13: Permiso denegado)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


In [ ]:
from PIL import Image, ImageTk
from tkinter import PhotoImage, messagebox
from clases.Arbol import Arbol
from clases.Mapa import Mapa
from clases.Uploader import Uploader
from clases.Alineador import Alineador
import tkinter as tk

lista_de_nombre_de_ubicaciones = ["Londres", "Paris", "Amsterdam", "Las Vegas", "Tokyo", "Buenos Aires", "San Pablo", "Tijuana", "Medellin", "Texas", "Nueva Orleans"]

class Handler():        
    
    def client_exit(self):
        root.destroy()
        
    def upload_fasta(self):        
        uploader = Uploader()
        if uploader.loadFasta():
            messagebox.showinfo(message="Se cargó el FASTA correctamente!", title="")
        else:
            messagebox.showinfo(message="No se cargó el FASTA", title="")
            
    def align_secuences(self):
        alineador = Alineador()
        alineador.alinear()
        
    def make_tree(self):
        arbol = Arbol()
        arbol.armarArbol()
        
    def make_map(self):
        mapa = Mapa()
        mapa.armar_mapa(lista_de_nombre_de_ubicaciones).render_mapa()

class BkgrFrame(tk.Frame):
    def __init__(self, parent, file_path, width, height):
        super(BkgrFrame, self).__init__(parent, borderwidth=0, highlightthickness=0)

        self.canvas = tk.Canvas(self, width=width, height=height)
        self.canvas.pack()

        pil_img = Image.open(file_path)
        self.img = ImageTk.PhotoImage(pil_img.resize((width, height), Image.ANTIALIAS))
        self.bg = self.canvas.create_image(0, 0, anchor=tk.NW, image=self.img)

    def add(self, widget, x, y):
        canvas_window = self.canvas.create_window(x, y, anchor=tk.NW, window=widget)
        return widget


if __name__ == '__main__':

    IMAGE_PATH = './img/MenuHerramientaBio.png'
    WIDTH, HEIGTH = 800, 600

    root = tk.Tk()
    root.geometry('{}x{}'.format(WIDTH, HEIGTH))
    
    handler = Handler()

    bkrgframe = BkgrFrame(root, IMAGE_PATH, WIDTH, HEIGTH)
    bkrgframe.pack()
    
    exitPhot = PhotoImage(file = r"./img/BotonSalir.png")
    fastaPhot = PhotoImage(file = r"./img/BotonFasta.png")
    alignPhot = PhotoImage(file = r"./img/BotonAlinear.png")
    treePhot = PhotoImage(file = r"./img/BotonArbolear.png")
    mapPhot = PhotoImage(file = r"./img/BotonMapear.png")

    button1 = bkrgframe.add(tk.Button(root, image=exitPhot, command= handler.client_exit), 650, 8)
    button2 = bkrgframe.add(tk.Button(root, image=fastaPhot, command= handler.upload_fasta), 310, 180)
    button3 = bkrgframe.add(tk.Button(root, image=alignPhot, command= handler.align_secuences), 65, 340)
    button4 = bkrgframe.add(tk.Button(root, image=treePhot, command= handler.make_tree), 65, 430)
    button5 = bkrgframe.add(tk.Button(root, image=mapPhot, command= handler.make_map), 60, 520)

    root.resizable(width=False, height=False)
    root.wm_attributes('-type', 'splash')
    root.mainloop()


In [ ]:
import os
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import IUPAC
import Bio.SeqIO as SeqIO
from Bio.Align.Applications import ClustalOmegaCommandline

ref_seq = "ERVVIGSKPFNEQYILANMIAILLEENGYKA"
target_seq = "ERVVIGSKPFNEQYILANMINGYKA"

if os.path.exists("./temp"):
    # Change the current working Directory
    owd = os.getcwd()
    os.chdir("./temp")
else:
    print("Can't change the Current Working Directory")    

in_file = "fasta.fasta"
out_file = "aligned.fasta"

clustalomega_cline = ClustalOmegaCommandline(infile=in_file, outfile=out_file, verbose=True, auto=True, force=True)
clustalomega_cline()

os.chdir(owd)
